In [ ]:
import pandas as pd, numpy as np, glob, os, math
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel(r"E:\Muthoot POC\Branch Data\State level data.xlsx")
df.head(2)

In [ ]:
df.shape

In [ ]:
 #df.info()

In [ ]:
df1 = df[['Loanno', 'Client_no', 'LoanAmount', 'Loanbalance', 'Paidupto',
       'Inst_complete', 'Centr', 'Group', 'LOAN_DATE', 'FIRSTINSTDT',
       'INTEREST', 'INSTALLMENTS', 'EWIAMT', 'LOANTYPE', 'Loancycle',
       'BRANCH',
       'MMLBRANCH', 'STATE']]

df1.columns

In [ ]:
df_loan_wise = df1

In [ ]:
df_loan_wise.describe()

In [ ]:
df2 = df_loan_wise[['BRANCH','Loanbalance']].groupby('BRANCH').count()
df2.columns = ['Productivity']

In [ ]:
df2

In [ ]:
df3 = df_loan_wise[['BRANCH','LoanAmount']].groupby('BRANCH').sum()
df2['Agg'] = df3['LoanAmount']
df2['Average Ticket Size'] = df2['Agg']/df2['Productivity']
df2.drop('Agg',axis=1,inplace=True)

In [ ]:
df2.head(2)

In [ ]:
# df2['Average DPD'] = df5['b']
# df2.head(2)

In [ ]:
df6 = df_loan_wise.groupby(['STATE','BRANCH']).agg(lambda x: x.value_counts().index[0])
df6.head(2)

In [ ]:
df6.to_csv(r"E:\Muthoot POC\Branch Data\aagain.csv")
df7 = pd.read_csv(r"E:\Muthoot POC\Branch Data\aagain.csv")

In [ ]:
df8 = df7[["STATE","BRANCH"]]
df9 = pd.merge(df2, df8, on=['BRANCH'])

In [ ]:
df9.to_csv(r"E:\Muthoot POC\Branch Data\Aggregated_Data1.csv")
df9.head(2)

In [ ]:
df4 = df_loan_wise[['BRANCH','Loanbalance']].groupby('BRANCH').sum()
df4['BRANCH'] = df4.index
df4.reset_index(drop=True, inplace=True)
df9['Book Size']=df4['Loanbalance']
df9.head()

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df9.describe()

In [ ]:
df9.head()

In [ ]:
df9['State Type'] = np.where(df9['Book Size'] > df9['Book Size'].quantile(q=0.66), 'Matured', 'New + Growing')
df9.to_csv(r"E:\Muthoot POC\Branch Data\New_BranchG+N.csv")
df9.head()

# Modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
df9.head()

In [ ]:
df9.to_csv(r"E:\Muthoot POC\Branch Data\New-Final.csv")
df_branch_wise = df9

In [ ]:
df_branch_wise.to_csv(r"E:\Muthoot POC\Branch Data\Branch_wise_modelling2.csv")
le = preprocessing.LabelEncoder()
df_branch_wise['State Type'] = le.fit_transform(df_branch_wise['State Type'])
le1= preprocessing.LabelEncoder()
df_branch_wise['STATE'] = le1.fit_transform(df_branch_wise['STATE'])

In [ ]:
df_state_dummy = pd.get_dummies(df_branch_wise['STATE'])

In [ ]:
df_branch_wise_2 = pd.concat([df_branch_wise, df_state_dummy], axis=1)

In [ ]:
df_branch_wise_2['State 1'] = df_branch_wise_2[0]
df_branch_wise_2['State 2'] = df_branch_wise_2[1]
df_branch_wise_2['State 3'] = df_branch_wise_2[2]

In [ ]:
df_branch_wise_1 = df_branch_wise_2[['Productivity', 'Average Ticket Size', 'State 1', 'State 2', 'State 3','Book Size',
                                      'State Type']]
df_branch_wise_1.head(3)

In [ ]:
df_branch_wise_1.info()

In [ ]:
df_branch_wise_1.columns
df_branch_2 = df_branch_wise_1[['Productivity','Average Ticket Size','State Type']]

In [ ]:
df_branch_2['State Type'].value_counts()

## Modelling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_branch_2.drop('State Type', axis=1), df_branch_2['State Type'], test_size=0.2, random_state=2,stratify=df_branch_2['State Type'])

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', 
                         multi_class='ovr').fit(X_train, y_train)

In [ ]:
clf

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(X_train)
model = sm.OLS(y_train, X).fit()
model.summary()

In [ ]:
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
df20 = pd.DataFrame(clf.predict_proba(X_test))

In [ ]:
df20 = df20.reset_index(drop=True)

In [ ]:
X_test = X_test.reset_index(drop=True)

In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
df_full1 = pd.concat([X_test,df20, y_test], axis=1)
df_full1.to_csv(r'E:\Muthoot POC\Branch Data\Outputs\df_new123.csv')

In [ ]:
#xxy = pd.concat([X_test, y_test], axis=1)

In [ ]:
#xxz.to_csv(r"E:\Muthoot POC\Branch Data\xxz.csv")

In [ ]:
#xxy.to_csv(r"E:\Muthoot POC\Branch Data\Y_TESTDATA.csv")